In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, accuracy_score
from datetime import datetime, timedelta

In [2]:
df = pd.read_csv("menstrual_cycle_dataset_with_factors.csv")

In [3]:
df.head(10)

,User ID,Age,BMI,Stress Level,Exercise Frequency,Sleep Hours,Diet,Cycle Start Date,Cycle Length,Period Length,Next Cycle Start Date,Symptoms
0,1,18,29.28,2,Moderate,5.4,Low Carb,2024-11-13 20:52:34.915012,26,7,2024-12-09 20:52:34.915012,Headache
1,1,18,29.28,2,Moderate,5.4,Low Carb,2024-12-09 20:52:34.915012,32,5,2025-01-10 20:52:34.915012,Fatigue
2,1,18,29.28,2,Moderate,5.4,Low Carb,2025-01-10 20:52:34.915012,41,7,2025-02-20 20:52:34.915012,Fatigue
3,1,18,29.28,2,Moderate,5.4,Low Carb,2025-02-20 20:52:34.915012,27,3,2025-03-19 20:52:34.915012,Fatigue
4,1,18,29.28,2,Moderate,5.4,Low Carb,2025-03-19 20:52:34.915012,42,5,2025-04-30 20:52:34.915012,Cramps
5,1,18,29.28,2,Moderate,5.4,Low Carb,2025-04-30 20:52:34.915012,41,5,2025-06-10 20:52:34.915012,Fatigue
6,1,18,29.28,2,Moderate,5.4,Low Carb,2025-06-10 20:52:34.915012,31,4,2025-07-11 20:52:34.915012,Headache
7,1,18,29.28,2,Moderate,5.4,Low Carb,2025-07-11 20:52:34.915012,48,7,2025-08-28 20:52:34.915012,Fatigue
8,1,18,29.28,2,Moderate,5.4,Low Carb,2025-08-28 20:52:34.915012,29,3,2025-09-26 20:52:34.915012,Fatigue
9,1,18,29.28,2,Moderate,5.4,Low Carb,2025-09-26 20:52:34.915012,47,4,2025-11-12 20:52:34.915012,Headache


In [4]:
df.columns

Index(['User ID', 'Age', 'BMI', 'Stress Level', 'Exercise Frequency',
       'Sleep Hours', 'Diet', 'Cycle Start Date', 'Cycle Length',
       'Period Length', 'Next Cycle Start Date', 'Symptoms'],
      dtype='object')

In [5]:
df['Symptoms'].value_counts()

Symptoms
Bloating       204
Fatigue        178
Cramps         177
Headache       175
Mood Swings    161
Name: count, dtype: int64

In [6]:
df["Stress Level"].value_counts()

Stress Level
4    227
5    184
1    183
2    158
3    143
Name: count, dtype: int64

In [7]:
df["Diet"].value_counts()

Diet
Vegetarian    268
Balanced      258
Low Carb      200
High Sugar    169
Name: count, dtype: int64

In [8]:
df['Exercise Frequency'].value_counts()

Exercise Frequency
High        361
Moderate    280
Low         254
Name: count, dtype: int64

In [9]:
df["Diet"].value_counts()

Diet
Vegetarian    268
Balanced      258
Low Carb      200
High Sugar    169
Name: count, dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895 entries, 0 to 894
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   User ID                895 non-null    int64  
 1   Age                    895 non-null    int64  
 2   BMI                    895 non-null    float64
 3   Stress Level           895 non-null    int64  
 4   Exercise Frequency     895 non-null    object 
 5   Sleep Hours            895 non-null    float64
 6   Diet                   895 non-null    object 
 7   Cycle Start Date       895 non-null    object 
 8   Cycle Length           895 non-null    int64  
 9   Period Length          895 non-null    int64  
 10  Next Cycle Start Date  895 non-null    object 
 11  Symptoms               895 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 84.0+ KB


In [11]:
df.isnull().sum()

User ID                  0
Age                      0
BMI                      0
Stress Level             0
Exercise Frequency       0
Sleep Hours              0
Diet                     0
Cycle Start Date         0
Cycle Length             0
Period Length            0
Next Cycle Start Date    0
Symptoms                 0
dtype: int64

In [12]:
df["Cycle Start Date"] = pd.to_datetime(df["Cycle Start Date"])
df["Next Cycle Start Date"] = pd.to_datetime(df["Next Cycle Start Date"])

In [13]:
df["Actual Cycle Length"] = (df["Next Cycle Start Date"] - df["Cycle Start Date"]).dt.days

In [14]:
print(df["Actual Cycle Length"].describe())
print(f"Number of unique cycle lengths: {df['Actual Cycle Length'].nunique()}")
print(f"Value counts:\n{df['Actual Cycle Length'].value_counts().sort_index()}")

count    895.000000
mean      37.373184
std        7.465008
min       25.000000
25%       31.000000
50%       37.000000
75%       43.000000
max       50.000000
Name: Actual Cycle Length, dtype: float64
Number of unique cycle lengths: 26
Value counts:
Actual Cycle Length
25    32
26    42
27    28
28    34
29    37
30    29
31    40
32    38
33    36
34    36
35    40
36    30
37    27
38    41
39    30
40    43
41    26
42    49
43    34
44    22
45    33
46    31
47    31
48    39
49    33
50    34
Name: count, dtype: int64


In [15]:
df["Cycle Start Month"] = df["Cycle Start Date"].dt.month
df["Cycle Start DayOfWeek"] = df["Cycle Start Date"].dt.dayofweek
df["Cycle Start DayOfYear"] = df["Cycle Start Date"].dt.dayofyear
df['Cycle Start Month_sin'] = np.sin(2 * np.pi * df['Cycle Start Month'] / 12)
df['Cycle Start Month_cos'] = np.cos(2 * np.pi * df['Cycle Start Month'] / 12)
df['Cycle_Length_Rolling_Avg'] = df.groupby('User ID')['Actual Cycle Length'].transform(
        lambda x: x.rolling(window=3, min_periods=1).mean()
    )
df['Is_Winter'] = df['Cycle Start Month'].isin([12, 1, 2]).astype(int)
df['Is_Spring'] = df['Cycle Start Month'].isin([3, 4, 5]).astype(int)
df['Is_Summer'] = df['Cycle Start Month'].isin([6, 7, 8]).astype(int)
df['Is_Fall'] = df['Cycle Start Month'].isin([9, 10, 11]).astype(int)


In [16]:
df["Age_Group"] = pd.cut(df["Age"], bins=[0, 20, 40, 50, 70], labels=[0, 1, 2, 7])

In [17]:
df["BMI_Category"] = pd.cut(df["BMI"], bins=[0, 18.5, 25, 70, 100], 
                           labels=["Underweight", "Normal", "Overweight", "Obese"])


In [18]:
label_cols = ["Exercise Frequency", "Diet", "Symptoms", "BMI_Category"]
label_encoders = {}

for col in label_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [19]:
feature_columns = [
    "Age", "BMI", "Stress Level", "Exercise Frequency",
    "Sleep Hours", "Diet", "Period Length", 
    "Symptoms", "Cycle Start Month", "Cycle Start DayOfWeek",
    "Age_Group", "BMI_Category","Cycle Start Month_sin", "Cycle Start Month_cos",
    "Cycle_Length_Rolling_Avg","Is_Winter", "Is_Spring", "Is_Summer", "Is_Fall"]

In [20]:
df.head()

,User ID,Age,BMI,Stress Level,Exercise Frequency,Sleep Hours,Diet,Cycle Start Date,Cycle Length,Period Length,...,Cycle Start DayOfYear,Cycle Start Month_sin,Cycle Start Month_cos,Cycle_Length_Rolling_Avg,Is_Winter,Is_Spring,Is_Summer,Is_Fall,Age_Group,BMI_Category
0,1,18,29.28,2,2,5.4,2,2024-11-13 20:52:34.915012,26,7,...,318,-5.000000e-01,8.660254e-01,26.000000,0,0,0,1,0,1
1,1,18,29.28,2,2,5.4,2,2024-12-09 20:52:34.915012,32,5,...,344,-2.449294e-16,1.000000e+00,29.000000,1,0,0,0,0,1
2,1,18,29.28,2,2,5.4,2,2025-01-10 20:52:34.915012,41,7,...,10,5.000000e-01,8.660254e-01,33.000000,1,0,0,0,0,1
3,1,18,29.28,2,2,5.4,2,2025-02-20 20:52:34.915012,27,3,...,51,8.660254e-01,5.000000e-01,33.333333,1,0,0,0,0,1
4,1,18,29.28,2,2,5.4,2,2025-03-19 20:52:34.915012,42,5,...,78,1.000000e+00,6.123234e-17,36.666667,0,1,0,0,0,1


In [21]:
X = df[feature_columns]
y = df["Actual Cycle Length"]

In [22]:
print(f"\n🎯 TARGET DISTRIBUTION:")
print(f"Target mean: {y.mean():.2f}")
print(f"Target std: {y.std():.2f}")
print(f"Target min: {y.min()}")
print(f"Target max: {y.max()}")


🎯 TARGET DISTRIBUTION:
Target mean: 37.37
Target std: 7.47
Target min: 25
Target max: 50


In [23]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [24]:
print(f"\n📊 DATA SPLIT:")
print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")


📊 DATA SPLIT:
Training set: 716 samples
Test set: 179 samples


In [25]:
numerical_features = ["Age", "BMI","Stress Level", "Sleep Hours", "Period Length"]
scaler = StandardScaler()

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[numerical_features] = scaler.fit_transform(X_train[numerical_features])
X_test_scaled[numerical_features] = scaler.transform(X_test[numerical_features])

In [26]:
def evaluate_models(X_train, X_test, y_train, y_test):
    """Evaluate multiple models and return the best one"""
    models = {
        'Linear Regression': LinearRegression(),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
        'SVR': SVR(kernel='rbf', C=1.0)
    }
    
    best_accuracy = 0
    best_model = None
    best_model_name = None
    
    print("🔍 MODEL COMPARISON:")
    print("=" * 50)
    
    for name, model in models.items():
        # Train model
        model.fit(X_train, y_train)
        
        # Predict
        y_pred = model.predict(X_test)
        
        # ✅ CORRECTED: Calculate accuracy within 7 days
        differences = np.abs(y_test - y_pred)
        accuracy_within_7_days = (differences <= 7).mean()  # Percentage within 7 days
        
        # Calculate MSE and average error
        mse = mean_squared_error(y_test, y_pred)
        avg_error = differences.mean()
        
        print(f"{name}:")
        print(f"  Accuracy within 7 days: {accuracy_within_7_days:.1%}")
        print(f"  MSE: {mse:.2f}")
        print(f"  Avg Error: {avg_error:.2f} days")
        print(f"  Max Error: {differences.max():.2f} days")
        print(f"  Min Error: {differences.min():.2f} days")
        
        # Track best model
        if accuracy_within_7_days > best_accuracy:
            best_accuracy = accuracy_within_7_days
            best_model = model
            best_model_name = name
    
    print("=" * 50)
    print(f"🏆 BEST MODEL: {best_model_name} with {best_accuracy:.1%} accuracy")
    
    return best_model, best_model_name

In [27]:

best_model, best_model_name = evaluate_models(X_train_scaled, X_test_scaled, y_train, y_test)

print("\n" + "="*60)
print("🔍 DETAILED DEBUG FOR BEST MODEL:")
print("="*60)


🔍 MODEL COMPARISON:
Linear Regression:
  Accuracy within 7 days: 80.4%
  MSE: 29.40
  Avg Error: 4.21 days
  Max Error: 14.93 days
  Min Error: 0.03 days
Random Forest:
  Accuracy within 7 days: 74.9%
  MSE: 37.97
  Avg Error: 4.82 days
  Max Error: 17.02 days
  Min Error: 0.13 days
SVR:
  Accuracy within 7 days: 79.9%
  MSE: 28.17
  Avg Error: 4.12 days
  Max Error: 14.05 days
  Min Error: 0.00 days
🏆 BEST MODEL: Linear Regression with 80.4% accuracy

🔍 DETAILED DEBUG FOR BEST MODEL:


In [38]:
def scale_numerical_features(df, scaler):
    """
    Scale numerical features using the trained scaler - FIXED VERSION
    """
    df = df.copy()
    
    numerical_features = ["Age", "BMI", "Stress Level", "Sleep Hours", "Period Length"]
    
    # Check if all numerical features are present
    missing_numerical = [f for f in numerical_features if f not in df.columns]
    if missing_numerical:
        return df
    
    # Extract ALL numerical features at once and scale them together
    numerical_data = df[numerical_features]
    scaled_data = scaler.transform(numerical_data)
    
    # Put the scaled values back into the DataFrame
    for i, feature in enumerate(numerical_features):
        df[feature] = scaled_data[:, i]
    
    return df

In [39]:
def generate_reason(row):
    """
    Analyze user parameters and provide reasons that may affect the menstrual cycle.
    """
    reasons = []
    age = row["Age"]
    if age < 18:
        reasons.append("Teen years often involve hormonal fluctuations as cycles establish")
    elif age >= 18 and age <= 25:
        reasons.append("Young adulthood typically has stable hormonal patterns")
    elif age > 25 and age <= 35:
        reasons.append("Prime reproductive years with generally stable hormones")
    elif age > 35 and age <= 45:
        reasons.append("Perimenopausal phase may cause hormonal variations")
    elif age > 45:
        reasons.append("Menopausal transition involves significant hormonal changes")

    # Stress (assuming scale 1-5)
    if row["Stress Level"] >= 4:
        reasons.append("High stress levels can disrupt hormonal balance")
    elif row["Stress Level"] <= 2:
        reasons.append("Low stress levels promote hormonal regularity")
    
    # Sleep
    if row["Sleep Hours"] < 7:
        reasons.append("Insufficient sleep may affect cycle regularity")
    elif row["Sleep Hours"] > 9:
        reasons.append("Excessive sleep could indicate hormonal imbalances")

    # BMI
    if row["BMI"] < 18.5:
        reasons.append("Low BMI may cause irregular periods")
    elif row["BMI"] > 25:
        reasons.append("High BMI can cause hormonal changes")

    # Exercise (assuming encoded: 0=Low, 1=Moderate, 2=High)
    if row["Exercise Frequency"] == 0:  # Low
        reasons.append("Low activity may lead to hormonal fluctuations")
    elif row["Exercise Frequency"] == 2:  # High
        reasons.append("High exercise intensity can affect cycle timing")

    # Diet (assuming encoded: 0=Vegetarian, 1=Balanced, 2=Low Carb, 3=High Sugar)
    if row["Diet"] == 3:  # High Sugar
        reasons.append("High sugar diet may impact hormonal balance")
    elif row["Diet"] == 1:  # Balanced
        reasons.append("Balanced diet supports reproductive health")

    if not reasons:
        return "Your health parameters appear stable and supportive of regular cycles."
    else:
        return "Factors that may influence your cycle:\n- " + "\n- ".join(reasons)

In [40]:
def generate_insights(predicted_days, actual_mean=28):
    """
    Provide meaningful interpretation of the prediction.
    """
    insights = []
    diff = predicted_days - actual_mean

    if diff > 7:
        insights.append(f"Predicted cycle ({predicted_days} days) is longer than average ({actual_mean} days)")
        insights.append("Longer cycles can be influenced by stress, diet, or hormonal changes")
    elif diff < -7:
        insights.append(f"Predicted cycle ({predicted_days} days) is shorter than average ({actual_mean} days)")
        insights.append("Shorter cycles may relate to lifestyle factors or natural variation")
    else:
        insights.append(f"Predicted cycle ({predicted_days} days) is within normal range")
        insights.append("Your cycle appears regular based on current health parameters")

    insights.append("Maintain consistent sleep, nutrition, and stress management for cycle regularity")
    return "\n".join(insights)

In [41]:
def encode_categorical_features(df, label_encoders):
    """
    Encode categorical features using the trained label encoders
    """
    df = df.copy()
    
    categorical_columns = ["Exercise Frequency", "Diet", "Symptoms", "BMI_Category"]
    
    for col in categorical_columns:
        if col in df.columns:
            try:
                # Get original value
                original_value = df[col].iloc[0]
                
                # Handle unseen labels
                if original_value not in label_encoders[col].classes_:
                    # Use the most common category as default
                    default_value = label_encoders[col].classes_[0]
                    df[col] = default_value
                else:
                    # Transform the value
                    df[col] = label_encoders[col].transform([original_value])[0]
                
                
            except Exception as e:
                # Fallback: use the first available category
                default_value = label_encoders[col].classes_[0]
                df[col] = label_encoders[col].transform([default_value])[0]
    
    return df

In [42]:
def apply_feature_engineering(df):
    """
    Apply all the same feature engineering used during training - FIXED VERSION
    """
    df = df.copy()
    
    # Ensure Cycle Start Date is datetime
    if not pd.api.types.is_datetime64_any_dtype(df['Cycle Start Date']):
        df['Cycle Start Date'] = pd.to_datetime(df['Cycle Start Date'])
    
    
    # Date features - ONLY use the ones that were in your training
    df['Cycle Start Month'] = df['Cycle Start Date'].dt.month
    df['Cycle Start DayOfWeek'] = df['Cycle Start Date'].dt.dayofweek
    


    
    # Age grouping
    df['Age_Group'] = pd.cut(df['Age'], bins=[0, 20, 30, 40, 50, 100], 
                            labels=[0, 1, 2, 3, 4])
    
    # BMI categorization
    conditions = [
        (df['BMI'] < 18.5),
        (df['BMI'] < 25),
        (df['BMI'] < 30),
        (df['BMI'] >= 30)
    ]
    choices = ["Underweight", "Normal", "Overweight", "Obese"]
    df['BMI_Category'] = np.select(conditions, choices, default="Normal")
    
    # Enhanced features (ONLY the ones that were in your training)
    df['Cycle Start Month_sin'] = np.sin(2 * np.pi * df['Cycle Start Month'] / 12)
    df['Cycle Start Month_cos'] = np.cos(2 * np.pi * df['Cycle Start Month'] / 12)
    df['Is_Winter'] = df['Cycle Start Month'].isin([12, 1, 2]).astype(int)
    df['Is_Spring'] = df['Cycle Start Month'].isin([3, 4, 5]).astype(int)
    df['Is_Summer'] = df['Cycle Start Month'].isin([6, 7, 8]).astype(int)
    df['Is_Fall'] = df['Cycle Start Month'].isin([9, 10, 11]).astype(int)
    
    # For single prediction, we can't calculate rolling average, so use a default

   

    
    return df

In [43]:
def predict_next_cycle(user_df, model, scaler, label_encoders):
    """
    Enhanced prediction function that works with the user DataFrame - FIXED VERSION
    """
    print(f"\n📊 Processing your input data...")
    
    # Apply all the same feature engineering as your training data
    user_df_processed = apply_feature_engineering(user_df)
    
    # Prepare features for prediction - MATCH EXACTLY what was used in training
    feature_columns = [
        "Age", "BMI", "Stress Level", "Exercise Frequency",
        "Sleep Hours", "Diet", "Period Length", 
        "Symptoms", "Cycle Start Month", "Cycle Start DayOfWeek",
        "Age_Group", "BMI_Category", "Cycle Start Month_sin", "Cycle Start Month_cos",
        "Cycle_Length_Rolling_Avg", "Is_Winter", "Is_Spring", "Is_Summer", "Is_Fall"
    ]
    
    # Check if all required features are present
    missing_features = [f for f in feature_columns if f not in user_df_processed.columns]
    if missing_features:
        print(f"❌ Missing features: {missing_features}")
        return None
    
    # Check for extra features that weren't in training
    extra_features = [f for f in user_df_processed.columns if f in user_df_processed.columns and f not in feature_columns and f not in ['Cycle Start Date', 'User ID']]
    if extra_features:
        print(f"⚠️  Extra features detected (will be ignored): {extra_features}")
    
    
    
    # Encode categorical variables
    user_df_encoded = encode_categorical_features(user_df_processed, label_encoders)
    
    # Scale numerical features
    user_df_scaled = scale_numerical_features(user_df_encoded, scaler)
    
    # Make prediction
    try:
        # Ensure we have the right features in the right order
        prediction_features = user_df_scaled[feature_columns]

        
        predicted_cycle_length = model.predict(prediction_features)[0]
        
        # Calculate next cycle date
        cycle_start_date = user_df_processed['Cycle Start Date'].iloc[0]
        predicted_next_date = cycle_start_date + timedelta(days=int(predicted_cycle_length))
        

        print("🎯 PREDICTION RESULTS")
        print(f"Predicted Cycle Length: {predicted_cycle_length:.1f} days")
        print(f"Predicted Next Cycle Start: {predicted_next_date.strftime('%Y-%m-%d')}")
        print(f"Confidence: 80% within ±7 days")
        print(f"Likely Range: {(predicted_next_date - timedelta(days=7)).strftime('%Y-%m-%d')} to {(predicted_next_date + timedelta(days=7)).strftime('%Y-%m-%d')}")

        row=user_df_processed.iloc[0]
        reasons = generate_reason(row)
        insights = generate_insights(predicted_cycle_length)
        print("\n🔍 HEALTH ANALYSIS")
        print(reasons)
        
        print("\n💡 INSIGHTS")
        print(insights)
        
        return predicted_cycle_length, predicted_next_date
        
    except Exception as e:
        import traceback
        return None

In [44]:
def create_user_dataframe(age, bmi, stress, exercise, diet, sleep, period_length, symptoms, cycle_start_str,avg_cycle_length):
    """
    Convert user input into a DataFrame with proper datetime conversion
    """
    # Convert date string to datetime FIRST
    cycle_start_date = pd.to_datetime(cycle_start_str)
    
    # Create DataFrame with the same structure
    user_df = pd.DataFrame({
        'Age': [age],
        'BMI': [bmi],
        'Stress Level': [stress],
        'Exercise Frequency': [exercise],
        'Sleep Hours': [sleep],
        'Diet': [diet],
        'Period Length': [period_length],
        'Symptoms': [symptoms],
        'Cycle Start Date': [cycle_start_date],
        "Cycle_Length_Rolling_Avg": [avg_cycle_length]
    })
    
    
    return user_df


In [45]:
def get_input():
    age = int(input("Enter Age: "))
    bmi = float(input("Enter BMI: "))
    stress = int(input("Enter Stress Level (1-5): "))
    exercise = input("Enter Exercise Frequency (Low, Moderate, High): ").strip().capitalize()
    diet = input("Enter Diet Type (Vegetarian, Balanced, Low Carb, High Sugar): ").strip().capitalize()
    sleep = float(input("Enter Average Sleep Hours: "))
    period_length = int(input("Enter Period Length (in days): "))
    symptoms = input("Enter Symptoms Severity (Bloating, Fatigue, Cramps, Headache, Mood Swings): ").strip().capitalize()
    avg_cycle_length=int(input("Enter your Average Cycle Length (in days): "))
    cycle_start_str = input("Enter Last Cycle Start Date (YYYY-MM-DD): ")
    

    # Convert to DataFrame with proper datetime conversion
    user_df = create_user_dataframe(
        age, bmi, stress, exercise, diet, sleep, 
        period_length, symptoms, cycle_start_str,avg_cycle_length
    )
    
    
    # Call prediction function
    predict_next_cycle(user_df, best_model, scaler, label_encoders)

In [46]:
get_input()


📊 Processing your input data...
🎯 PREDICTION RESULTS
Predicted Cycle Length: 40.5 days
Predicted Next Cycle Start: 2025-06-21
Confidence: 80% within ±7 days
Likely Range: 2025-06-14 to 2025-06-28

🔍 HEALTH ANALYSIS
Factors that may influence your cycle:
- Young adulthood typically has stable hormonal patterns
- High stress levels can disrupt hormonal balance

💡 INSIGHTS
Predicted cycle (40.46347762501156 days) is longer than average (28 days)
Longer cycles can be influenced by stress, diet, or hormonal changes
Maintain consistent sleep, nutrition, and stress management for cycle regularity


In [47]:
import pickle
with open("best_model.pkl", "wb") as f:
    pickle.dump(best_model, f)
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)
with open("label_encoders.pkl", "wb") as f:
    pickle.dump(label_encoders, f)
